In [ ]:
#드라이브 사용한다고 선언
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#zip 압축해제를 위한 부분
#cd "/content/drive/MyDrive/tmp"
!unzip -qq "/content/drive/MyDrive/tmp/crawling_phot.zip"

In [ ]:
from PIL import Image
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split

# 분류 대상 카테고리 선택하기 
photo_dir = "/content/drive/MyDrive/tmp/image"
categories = ["apartment","beach","cherry_blossoms","city","forest","sunset"]
nb_classes = len(categories)

In [ ]:
# 이미지 크기 지정 
image_w = 64 
image_h = 64
pixels = image_w * image_h * 3
# 이미지 데이터 읽어 들이기 
X = []
Y = []
for idx, cat in enumerate(categories):
    # 레이블 지정 
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    # 이미지 
    image_dir = photo_dir + "/" + cat
    #image_dir = photo_dir
    files = glob.glob(image_dir+"/*.jpg")
    for i, f in enumerate(files):
        img = Image.open(f) 
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)      # numpy 배열로 변환
        X.append(data)
        Y.append(label)
        if i % 10 == 0:
            print(i, "\n", data)
X = np.array(X)
Y = np.array(Y)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
490 
 [[[119 121 111]
  [119 121 111]
  [123 125 115]
  ...
  [106 112 111]
  [106 111 110]
  [110 115 113]]

 [[ 54  57  40]
  [ 53  56  40]
  [ 59  62  45]
  ...
  [ 33  41  39]
  [ 32  40  38]
  [ 38  47  44]]

 [[ 63  66  46]
  [ 61  65  46]
  [ 65  69  49]
  ...
  [ 40  48  46]
  [ 39  47  45]
  [ 46  54  52]]

 ...

 [[ 14   4   0]
  [  6   2   2]
  [  3   2   1]
  ...
  [ 16  11   1]
  [ 19  13   1]
  [ 27  22  10]]

 [[ 20   5   0]
  [  5   1   1]
  [  2   2   1]
  ...
  [ 15  13   4]
  [ 18  14   4]
  [ 27  24  14]]

 [[ 30  12   4]
  [  9   3   2]
  [  2   2   1]
  ...
  [ 21  19  11]
  [ 20  18   8]
  [ 27  25  15]]]
500 
 [[[ 74  73  80]
  [ 44  44  57]
  [ 47  46  69]
  ...
  [ 67  56  88]
  [ 60  53  86]
  [ 53  47  80]]

 [[ 36  33  46]
  [  0   0  14]
  [  1   0  27]
  ...
  [ 24  16  55]
  [ 16  11  51]
  [  7   4  45]]

 [[ 42  37  53]
  [  6   0  23]
  [ 10   0  33]
  ...
  [ 25  27  64]
  [ 18  21  61]
  [ 11  13  54]]

 ...

 [[ 

In [ ]:
# 학습 전용 데이터와 테스트 전용 데이터 구분 
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.33,random_state=42)
xy = (X_train, X_test, y_train, y_test)

print('>>> data 저장중 ...')
np.save("/content/drive/MyDrive/tmp/7obj.npy", xy)
print("ok,", len(Y))

>>> data 저장중 ...


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


ok, 11205


In [ ]:
from keras.models import Sequential
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
import os

# 카테고리 지정하기
categories = ["apartment","beach","cherry_blossoms","city","forest","sunset"]
nb_classes = len(categories)
# 이미지 크기 지정하기
image_w = 64
image_h = 64
# 데이터 열기 
#X_train, X_test, y_train, y_test = np.load("./image/7obj.npy")
# 데이터 정규화하기(0~1사이로)
X_train = X_train.astype("float") / 256
X_test  = X_test.astype("float")  / 256
print('X_train shape:', X_train.shape)

# 모델 구조 정의 
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

X_train shape: (7507, 64, 64, 3)


In [ ]:
# 전결합층
model.add(Flatten())    # 벡터형태로 reshape
model.add(Dense(512))   # 출력
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))
# 모델 구축하기
model.compile(loss='categorical_crossentropy',   # 최적화 함수 지정
    optimizer='rmsprop',
    metrics=['accuracy'])
# 모델 확인
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_4 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 64)       

In [ ]:
model.fit(X_train, y_train, batch_size=53, epochs=30)

Epoch 1/30
142/142 [==============================] - 68s 469ms/step - loss: 2.3544 - accuracy: 0.3162
Epoch 2/30
142/142 [==============================] - 67s 471ms/step - loss: 1.1988 - accuracy: 0.5652
Epoch 3/30
142/142 [==============================] - 67s 472ms/step - loss: 1.0004 - accuracy: 0.6425
Epoch 4/30
142/142 [==============================] - 68s 481ms/step - loss: 0.8567 - accuracy: 0.6913
Epoch 5/30
142/142 [==============================] - 67s 472ms/step - loss: 0.7649 - accuracy: 0.7238
Epoch 6/30
142/142 [==============================] - 67s 473ms/step - loss: 0.6879 - accuracy: 0.7494
Epoch 7/30
142/142 [==============================] - 67s 474ms/step - loss: 0.5953 - accuracy: 0.7926
Epoch 8/30
142/142 [==============================] - 67s 475ms/step - loss: 0.4849 - accuracy: 0.8256
Epoch 9/30
142/142 [==============================] - 67s 474ms/step - loss: 0.4206 - accuracy: 0.8497
Epoch 10/30
142/142 [==============================] - 68s 475ms/step - l

In [ ]:
# 학습 완료된 모델 저장
import h5py
hdf5_file = "/content/drive/MyDrive/tmp/7obj-model.hdf5"

In [ ]:
#실행 NO

if os.path.exists(hdf5_file):
    # 기존에 학습된 모델 불러들이기
    model.load_weights(hdf5_file)
else:
    # 학습한 모델이 없으면 파일로 저장
    model.fit(X_train, y_train, batch_size=32, nb_epoch=10)
    model.save_weights(hdf5_file)

In [ ]:
# 모델 평가하기 
score = model.evaluate(X_test, y_test)
print('loss=', score[0])        # loss
print('accuracy=', score[1])    # acc

116/116 [==============================] - 10s 79ms/step - loss: 2.3142 - accuracy: 0.7171
loss= 2.3141515254974365
accuracy= 0.7171444296836853


In [ ]:
from PIL import Image
# 적용해볼 이미지 
test_image = '/content/drive/MyDrive/tmp/image/forest/forest109.jpg'
# 이미지 resize
img = Image.open(test_image)
img = img.convert("RGB")
img = img.resize((64,64))
data = np.asarray(img)
X = np.array(data)
X = X.astype("float") / 256
X = X.reshape(-1, 64, 64,3)
# 예측
pred = model.predict(X)  
result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
print('New data category : ',categories[result[0]])


New data category :  forest


In [ ]:
from PIL import Image
# 적용해볼 이미지 
test_image1 = '/content/drive/MyDrive/tmp/image/apartment/아파트1100.jpg'
# 이미지 resize
img = Image.open(test_image1)
img = img.convert("RGB")
img = img.resize((64,64))
data = np.asarray(img)
X = np.array(data)
X = X.astype("float") / 256
X = X.reshape(-1, 64, 64,3)
# 예측
pred = model.predict(X)  
result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
print('New data category : ',categories[result[0]])


New data category :  apartment


In [ ]:
from PIL import Image
# 적용해볼 이미지 
test_image2 = '/content/drive/MyDrive/tmp/image/beach/beach1100.jpg'
# 이미지 resize
img = Image.open(test_image2)
img = img.convert("RGB")
img = img.resize((64,64))
data = np.asarray(img)
X = np.array(data)
X = X.astype("float") / 256
X = X.reshape(-1, 64, 64,3)
# 예측
pred = model.predict(X)  
result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
print('New data category : ',categories[result[0]])


New data category :  beach


In [ ]:
from PIL import Image
# 적용해볼 이미지 
test_image3 = '/content/drive/MyDrive/tmp/image/cherry_blossoms/벚꽃1101.jpg'
# 이미지 resize
img = Image.open(test_image3)
img = img.convert("RGB")
img = img.resize((64,64))
data = np.asarray(img)
X = np.array(data)
X = X.astype("float") / 256
X = X.reshape(-1, 64, 64,3)
# 예측
pred = model.predict(X)  
result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
print('New data category : ',categories[result[0]])


New data category :  cherry_blossoms


In [ ]:
from PIL import Image
# 적용해볼 이미지 
test_image4 = '/content/drive/MyDrive/tmp/image/city/city1114.jpg'
# 이미지 resize
img = Image.open(test_image4)
img = img.convert("RGB")
img = img.resize((64,64))
data = np.asarray(img)
X = np.array(data)
X = X.astype("float") / 256
X = X.reshape(-1, 64, 64,3)
# 예측
pred = model.predict(X)  
result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
print('New data category : ',categories[result[0]])


New data category :  city


In [ ]:
from PIL import Image
# 적용해볼 이미지 
test_image5 = '/content/drive/MyDrive/tmp/image/sunset/일몰1101.jpg'
# 이미지 resize
img = Image.open(test_image5)
img = img.convert("RGB")
img = img.resize((64,64))
data = np.asarray(img)
X = np.array(data)
X = X.astype("float") / 256
X = X.reshape(-1, 64, 64,3)
# 예측
pred = model.predict(X)  
result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
print('New data category : ',categories[result[0]])


New data category :  sunset
